In [1]:
import tensorflow as tf
import librosa.display
import matplotlib.pyplot as plt
import numpy as np
import os
import random
import timeit

In [2]:
filenames = []
for file in os.listdir('audio'):
    filenames.append(file)
# filenames[0:2]

In [19]:
random.shuffle(filenames)

In [20]:
max, min = 76117, 53310
start, end = 21000, 53310
wave_length = end-start
num_train = int(len(filenames)*0.7)
num_test = len(filenames) - num_train

In [35]:
data_x_train = []
data_x_test = []

start_time = timeit.default_timer()
for filename in filenames[:num_train]:
    data, sampling_rate = librosa.load("audio/" + filename, res_type='kaiser_fast')
    data_x_train.append(data[start:end])
    
for filename in filenames[num_train:]:
    data, sampling_rate = librosa.load("audio/" + filename, res_type='kaiser_fast')
    data_x_test.append(data[start:end])
    
stop_time = timeit.default_timer()
print('runtime: ', stop_time - start_time)  

runtime:  12.478960180991976


In [22]:
# max = 0
# min = 57117
# for i in range(len(data_x)):
#     if len(data_x[i]) > max:
#         max = len(data_x[i])
#     if len(data_x[i]) < min:
#         min = len(data_x[i])
print("max =", max, ",min =", min, ",length =",wave_length, ",len(data_x) =",len(data_x))

max = 76117 ,min = 53310 ,length = 32310 ,len(data_x) = 128


In [41]:
data_y_train = []
data_y_test = []

for filename in filenames[:num_train]:
    if filename[7] == '3':
        data_y_train.append(1)
    elif filename[7] == '4':
        data_y_train.append(0)
        
for filename in filenames[num_train:]:
    if filename[7] == '3':
        data_y_test.append(1)
    elif filename[7] == '4':
        data_y_test.append(0)   

In [49]:
# takes every value in data_y to an array
# ex: [0, 1, 1, 1, 0] => [[0], [1], [1], [1], [0]]
data_y_train_correct = []
data_y_test_correct = []

for val in data_y_train:
    val_arr = []
    val_arr.append(val)
    data_y_train_correct.append(val_arr)
    
for val in data_y_test:
    val_arr = []
    val_arr.append(val)
    data_y_test_correct.append(val_arr)

In [58]:
data_xy_train = []
for i in range(len(data_x_train)):
    data_xy_train.append( (data_x_train[i], data_y_train_correct[i]) )

In [89]:
def getXvalues(data_xy):
    x_values = []
    for data in data_xy:
        x_values.append(data[0])
    return x_values

def getYvalues(data_xy):
    y_values = []
    for data in data_xy:
        y_values.append(data[1])
    return y_values

In [52]:
def logistic_fun(z):
    return 1/(1.0 + np.exp(-z))

In [96]:
data_xy_train

[(array([ 1.0684992e-06,  3.7056247e-06,  1.1272429e-05, ...,
         -1.0291030e-02, -7.4613965e-03,  9.4495277e-04], dtype=float32), [1]),
 (array([ 5.9187016e-07, -1.5703230e-07, -9.2744628e-08, ...,
          4.8424816e-03,  4.8939311e-03,  4.7554560e-03], dtype=float32), [0]),
 (array([ 5.8532267e-05,  8.2692161e-05,  5.9512797e-05, ...,
         -6.4005964e-03, -9.2632165e-03, -9.2809070e-03], dtype=float32), [1]),
 (array([ 1.2292220e-04,  7.7097095e-05, -8.1462080e-05, ...,
         -2.4215721e-02, -3.2661915e-02, -4.9243528e-02], dtype=float32), [1]),
 (array([ 2.9210269e-05, -3.0221156e-07, -3.7230816e-06, ...,
         -2.2239949e-05, -3.5313555e-05, -9.9485123e-06], dtype=float32), [0]),
 (array([-1.20815705e-04, -1.15397888e-04, -6.91200739e-06, ...,
         -2.01451063e-01, -2.24132150e-01, -1.98831007e-01], dtype=float32),
  [0]),
 (array([-0.00015954, -0.00011426, -0.0001213 , ...,  0.00730817,
          0.0221633 ,  0.02262837], dtype=float32), [1]),
 (array([ 1.5061

In [93]:
features = wave_length
eps = 1e-12

x = tf.placeholder(tf.float32, [None, features])

y_ = tf.placeholder(tf.float32, [None, 1])

W = tf.Variable(tf.zeros([features,1]))

b = tf.Variable(tf.zeros([1]))

y = 1 / (1.0 + tf.exp(-(tf.matmul(x,W) + b)))

loss1 = -(y_ * tf.log(y + eps) + (1 - y_) * tf.log(1 - y + eps))

loss = tf.reduce_mean(loss1)

update = tf.train.GradientDescentOptimizer(0.00001).minimize(loss)

sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(0,100):
    sess.run(update, feed_dict = {x:getXvalues(data_xy_train), y_:getYvalues(data_xy_train)}) #BGD
    random.shuffle(data_xy_train)
    

In [29]:
# # guessing random number from 0 to 1, ~50% success
# # our model should be at least as good as this!!!!
# right = 0
# last_x = len(data_x)
# for i in range(num_train, last_x):
#     pred = random.randint(0,1)
#     if data_y[i] == 0 and pred < 0.5:
#         right += 1
#     if data_y[i] == 1 and pred > 0.5:
#         right += 1
#     print("pred:",pred,"true:",data_y[i],"right:",right)

In [94]:
# data_x_train =          [arr1, arr2, arr3]       # data_x_test = [arr4, arr5]
# shuffle(data_x_train) = [arr2, arr3, arr1]
# data_y_train = [0, 1, 1] # data_y_test = [0, 1]

right = 0
for i in range(len(data_x_test)):
    pred = logistic_fun(np.matmul(data_x_test[i],sess.run(W)) + sess.run(b))
    if data_y_test[i] == 0 and pred < 0.5:
        right += 1
    if data_y_test[i] == 1 and pred > 0.5:
        right += 1
    print("pred:",pred,"true:",data_y_test[i],"right:",right)

pred: [0.4999981] true: 0 right: 1
pred: [0.49999726] true: 0 right: 2
pred: [0.49999917] true: 0 right: 3
pred: [0.49999505] true: 0 right: 4
pred: [0.4999801] true: 1 right: 4
pred: [0.4999963] true: 0 right: 5
pred: [0.49999887] true: 1 right: 5
pred: [0.49999642] true: 1 right: 5
pred: [0.4999991] true: 1 right: 5
pred: [0.50000185] true: 1 right: 6
pred: [0.49999988] true: 1 right: 6
pred: [0.4999889] true: 0 right: 7
pred: [0.49999833] true: 0 right: 8
pred: [0.4999975] true: 0 right: 9
pred: [0.49999893] true: 1 right: 9
pred: [0.5000012] true: 1 right: 10
pred: [0.4999985] true: 1 right: 10
pred: [0.4999982] true: 1 right: 10
pred: [0.49999678] true: 0 right: 11
pred: [0.49999827] true: 1 right: 11
pred: [0.49999774] true: 0 right: 12
pred: [0.49999845] true: 0 right: 13
pred: [0.49999797] true: 1 right: 13
pred: [0.49999893] true: 0 right: 14
pred: [0.50000995] true: 1 right: 15
pred: [0.500008] true: 0 right: 15
pred: [0.49999785] true: 1 right: 15
pred: [0.5000041] true: 0 r

In [95]:
#with happy/sad files from first 4 actors we got 6 out of 20 correct, 30%
#with happy/sad files from first 8 actors we got 15 out of 39 correct, 38.46153846153847%
print("right:",right,"total:",num_test,"percent correct:",right/num_test*100)

right: 21 total: 39 percent correct: 53.84615384615385
